<a href="https://colab.research.google.com/github/noaihere/encyclepedia_python/blob/main/classmethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following example from https://stackabuse.com/pythons-classmethod-and-staticmethod-explained/

In [1]:
class Student(object):

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    @classmethod
    def from_string(cls, name_str):
        first_name, last_name = map(str, name_str.split(' '))
        student = cls(first_name, last_name)
        return student

In [2]:
scott = Student.from_string('Scott Robinson')

scott.first_name

'Scott'

Following example from https://djangocentral.com/classmethod-and-staticmethod-explained/

In [3]:
class MyClass():
    
    TOTAL_OBJECTS=0
    
    def __init__(self):
        MyClass.TOTAL_OBJECTS = MyClass.TOTAL_OBJECTS+1
       
    @classmethod
    def total_objects(cls):
        print("Total objects: ",cls.TOTAL_OBJECTS)

# Creating objects of parent class       
my_obj1 = MyClass()
my_obj2 = MyClass()

MyClass.total_objects()

# Creating a child class
class ChildClass(MyClass):
    TOTAL_OBJECTS=3
    pass

my_classobj1 = ChildClass()
my_classobj2 = ChildClass()
ChildClass.total_objects()

Total objects:  2
Total objects:  3


In [4]:
class Student(object):

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    @classmethod
    def from_string(cls, name_str):
        first_name, last_name = map(str, name_str.split(' '))
        student = cls(first_name, last_name)
        return student

class Person(Student):
    def add_name(self):
        return self.first_name+' test'

tim = Person.from_string('Tim Robinson')

print(tim.first_name)
tim.add_name()

Tim


'Tim test'

Following example taken from https://www.informit.com/articles/article.aspx?p=2992597&seqNum=3

In [5]:
import os

In [6]:
# class for input data

class GenericInputData:

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

In [7]:
# subclass of GenericInputData

class PathInputData(GenericInputData):

    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
         with open(self.path) as f:
             return f.read()
        
    @classmethod
    def generate_inputs(cls, config):
         data_dir = config['data_dir']
         for name in os.listdir(data_dir):
              if name.endswith('txt'):
                   yield cls(os.path.join(data_dir, name))

In [8]:
# class to create workers

class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

In [9]:
# subclass of GenericWorker
# read is method of PathInputData class

class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')

In [10]:
# execute function

def execute(workers):
    result_list=[]  
    for w in workers:
        w.map()
        result_list.append(w.result)
    print(result_list)    


In [11]:
# create 2 txt files as input data

with open('/content/Example1.txt', 'w') as writefile:
    writefile.write("This is line A\n")

with open('/content/Example2.txt', 'w') as writefile:
    writefile.write("This is line A\n")
    writefile.write("This is line B\n")
    writefile.write("This is line B\n")
    writefile.write("This is line B\n")            

In [12]:
# get files directory

!pwd
os.listdir('/content')[1].endswith('txt')

/content


False

In [13]:
# mapreduce function

def mapreduce(worker_class, input_class, config):
     workers = worker_class.create_workers(input_class, config)
     return execute(workers)

In [14]:
 # get final result
 
 config = {'data_dir':'/content'}
 mapreduce(LineCountWorker, PathInputData, config)

[4, 1]


In [15]:
# create json input file

import json

data = {
    "text": 
        'This is line A\n This is line A\n'
    }

with open("/content/Example1.json", "w") as write_file:
    json.dump(data, write_file)

In [16]:
# create different subclass of GenericInputData

class JsonInputData(GenericInputData):

    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
         with open(self.path) as f:
             return f.read()
        
    @classmethod
    def generate_inputs(cls, config):
         data_dir = config['data_dir']
         for name in os.listdir(data_dir):
              if name.endswith('json'):
                   yield cls(os.path.join(data_dir, name))

In [17]:
# create different subclass of GenericWorker

class JsonCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = json.loads(data)['text'].count('\n')

In [18]:
 # get final result using same function but different subclasses
 
 config = {'data_dir':'/content'}
 mapreduce(JsonCountWorker, JsonInputData, config)

[2]
